In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import glob
import tqdm


In [ ]:
import PIL
import numpy as np
from PIL import Image
from numpy import asarray
# Open the image form working directory
from tqdm import tqdm



In [ ]:
def getData(num_samples=500,PATHS=['../../datasets/cells/all/*bmp','../../datasets/cells/hem/*bmp']):
    count=0
    FILES={}
    LEN=np.zeros(2,)
    for i in PATHS:
        FILES[count]=glob.glob(i)
        LEN[count]=len(FILES[count])
        count=count+1
    print('Number in each class: ',LEN)

    INDEX_ARRAY={}
    for i in np.arange(len(PATHS)):
        INDEX_ARRAY[i]=np.random.permutation(int(LEN[i]))

    print(INDEX_ARRAY[0])

    NUM_TRAINING=10
    TOTAL_NUMBER_OF_EXAMPLES=np.floor(np.array(LEN).astype(int)*.1).astype(int)
    TOTAL_NUMBER_OF_VALIDATION_EXAMPLES=np.array(num_samples-TOTAL_NUMBER_OF_EXAMPLES).astype(int)
    print(TOTAL_NUMBER_OF_EXAMPLES,TOTAL_NUMBER_OF_VALIDATION_EXAMPLES)

    ar_tr = np.zeros([TOTAL_NUMBER_OF_EXAMPLES.sum(),450,450, 3], dtype=np.uint8)
    label_tr=np.empty((TOTAL_NUMBER_OF_EXAMPLES.sum(),), dtype='uint8')
    ar_ts = np.zeros([TOTAL_NUMBER_OF_VALIDATION_EXAMPLES.sum(),450,450, 3], dtype=np.uint8)
    label_ts=np.empty((TOTAL_NUMBER_OF_VALIDATION_EXAMPLES.sum(),), dtype='uint8')

    num=0
    for i in np.arange(len(PATHS)):
        for f in tqdm(np.array(FILES[i])[INDEX_ARRAY[i][:TOTAL_NUMBER_OF_EXAMPLES[i]]]):
            image = Image.open(f)
            data = asarray(image)
            ar_tr[num,:,:,:]=data
            label_tr[num]=i
            num=num+1
    plt.figure()
    plt.imshow(ar_tr[2,:,:,0])
    plt.figure()
    plt.imshow(ar_tr[TOTAL_NUMBER_OF_EXAMPLES[0]+2,:,:,0]);    
    
    
    num=0
    for i in np.arange(len(PATHS)):
        for f in tqdm(np.array(FILES[i])[INDEX_ARRAY[i][TOTAL_NUMBER_OF_EXAMPLES[i]:num_samples]]):
            image = Image.open(f)
            data = asarray(image)
            ar_ts[num,:,:,:]=data
            label_ts[num]=i
            num=num+1
    plt.figure()
    plt.imshow(ar_ts[2,:,:,0])
    plt.figure()
    plt.imshow(ar_ts[TOTAL_NUMBER_OF_VALIDATION_EXAMPLES[0]+2,:,:,0]);    
    
    
    
    x_train=ar_tr
    x_test=ar_ts
    y_train=label_tr
    y_test=label_ts
    return (x_train, y_train), (x_test, y_test)

In [ ]:
(train_images, train_labels), (test_images, test_labels)=getData(500)

In [ ]:
print(train_images.shape,len(train_labels))
print(test_images.shape,len(test_labels))

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
class_names = ['abnormal','normal']



In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(450, 450, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3,activation=tf.nn.softmax))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=3, 
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
history.history